In [1]:
!pip install opencv-python
!pip install opencv-contrib-python
!pip install numpy

In [2]:
import os 
import cv2 as cv
import numpy as np

### 2.1. Training the algorithm

In [3]:
DIR = 'faces/train'
people = [person for person in os.listdir(DIR) if os.path.isdir(os.path.join(DIR, person))]
print(people)
# direction to the folder with images


haar = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

features = []
labels = []

def create_train():
    for person in people:
        path = os.path.join(DIR, person)
        label = people.index(person)

        for img in os.listdir(path):
            img_path = os.path.join(path, img)

            img_array = cv.imread(img_path)
            gray = cv.cvtColor(img_array, cv.COLOR_BGR2GRAY)

            faces_rect = haar.detectMultiScale(gray)

            for (x, y, w, h) in faces_rect:
                faces_roi = gray[y:y+h, x:x+w]
                features.append(faces_roi)
                labels.append(label)

['Gates', 'Jobs', 'Musk']


In [4]:
import os
import cv2 as cv

DIR = 'faces/train'
people = [person for person in os.listdir(DIR) if os.path.isdir(os.path.join(DIR, person))]
print(people)

haar = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

features = []
labels = []

def create_train():
    for person in people:
        path = os.path.join(DIR, person)
        label = people.index(person)

        for img in os.listdir(path):
            img_path = os.path.join(path, img)

            img_array = cv.imread(img_path)
            print(img_path)
            gray = cv.cvtColor(img_array, cv.COLOR_BGR2GRAY)

            faces_rect = haar.detectMultiScale(gray)

            # Delete the image if no faces or more than one face are detected
            if len(faces_rect) != 1:
                print(f"Deleting {img_path} - Number of faces detected: {len(faces_rect)}")
                os.remove(img_path)
                continue

            for (x, y, w, h) in faces_rect:
                faces_roi = gray[y:y+h, x:x+w]
                features.append(faces_roi)
                labels.append(label)

create_train()

['Gates', 'Jobs', 'Musk']
faces/train\Gates\0x0.jpg
Deleting faces/train\Gates\0x0.jpg - Number of faces detected: 7
faces/train\Gates\104461165-Bill_Gates_green.jpg
faces/train\Gates\Bill-Gates-2011.jpg
Deleting faces/train\Gates\Bill-Gates-2011.jpg - Number of faces detected: 5
faces/train\Gates\billgatestock.jpg
Deleting faces/train\Gates\billgatestock.jpg - Number of faces detected: 5
faces/train\Gates\Bill_Gates_2017_(cropped).jpg
faces/train\Gates\Bill_Gates_July_2014.jpg
Deleting faces/train\Gates\Bill_Gates_July_2014.jpg - Number of faces detected: 2
faces/train\Gates\download (1).jpg
faces/train\Gates\download (2).jpg
faces/train\Gates\download (3).jpg
faces/train\Gates\download.jpg
faces/train\Gates\GettyImages-187969754_441409_w6ga1c.jpg
faces/train\Gates\images (1).jpg
faces/train\Gates\images.jpg
faces/train\Gates\summer-books_2020_article-hero_1200x564_01.jpg
Deleting faces/train\Gates\summer-books_2020_article-hero_1200x564_01.jpg - Number of faces detected: 0
faces/trai

In [5]:
create_train()

print('НАТРЕНИРОВАНО! ---------------')

features = np.array(features, dtype='object')
labels = np.array(labels)

face_recognizer = cv.face.LBPHFaceRecognizer_create()

# Train the Recognizer on the features list and the labels list
face_recognizer.train(features,labels)

face_recognizer.save('face_trained.yml')
np.save('features.npy', features)
np.save('labels.npy', labels)

faces/train\Gates\104461165-Bill_Gates_green.jpg
faces/train\Gates\Bill_Gates_2017_(cropped).jpg
faces/train\Gates\download (1).jpg
faces/train\Gates\download (2).jpg
faces/train\Gates\download (3).jpg
faces/train\Gates\download.jpg
faces/train\Gates\GettyImages-187969754_441409_w6ga1c.jpg
faces/train\Gates\images (1).jpg
faces/train\Gates\images.jpg
faces/train\Gates\thumbs_b_c_3bbbdbfe17acd1149ac4688bfdc372c5.jpg
faces/train\Jobs\03stewart-superJumbo-v2.jpg
faces/train\Jobs\6203cabebd504100191957d6.webp
faces/train\Jobs\800px-Steve_Jobs.jpg
faces/train\Jobs\apple_jobs_8450869_custom-fcad095f883b744792dc0b2aa6347b9ad32c9b48-s1100-c50.jpg
faces/train\Jobs\Gira_internacional_USA_-_Steve_Jobs_(cropped).jpg
faces/train\Jobs\images (1).jpg
faces/train\Jobs\images.jpg
faces/train\Jobs\maxresdefault.jpg
faces/train\Jobs\s6XnBUcLS4kGegcn.jpg
faces/train\Jobs\steve-jobs-2-1.webp
faces/train\Musk\20190318015843-1fd9c042.jpg
faces/train\Musk\52397a38565a0.jpg
faces/train\Musk\960x0.jpg
faces/tra

### 2.2. Face Recognition

In [5]:
import os
import numpy as np
import cv2

# Directory to the folder with images
DIR = 'faces/train'

# Load the Haar cascade for face detection
haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Dynamically create the list of people from folder names
people = [person for person in os.listdir(DIR) if os.path.isdir(os.path.join(DIR, person))]

# Load the pre-trained LBPH face recognizer
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.read('face_trained.yml')

# Initialize counters for accuracy calculation
total_faces = 0
correct_predictions = 0

# Function to classify a face and return the predicted label and confidence
def Classify(face):
    gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    label, confidence = face_recognizer.predict(gray)
    
    # Check if the label is within the valid range
    if label >= 0 and label < len(people):
        predicted_person = people[label]
    else:
        predicted_person = "Unknown"
    
    return predicted_person, confidence

# Initialize video capture from webcam (change 0 to another number if using a different camera)
cap = cv2.VideoCapture(0)
cap.set(3, 640)  # Set Width
cap.set(4, 480)  # Set Height

while True:
    ret, img = cap.read()
    if not ret:
        break

    img = cv2.flip(img, 1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces in the video frame
    faces = haar_cascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=5,
        minSize=(20, 20)
    )

    for (x, y, w, h) in faces:
        face = img[y:y+h, x:x+w]

        # Classify the detected face
        predicted_person, confidence = Classify(face)

        # Check if prediction confidence is within a reasonable threshold
        if confidence < 100:  # Adjust this threshold as needed
            correct_predictions += 1

        total_faces += 1

        # Display the prediction and bounding box around the face
        cv2.putText(img, f'{predicted_person} ({confidence:.2f})', (x, y - 10), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 2)
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Calculate and display accuracy
    if total_faces > 0:
        accuracy = (correct_predictions / total_faces) * 100
        cv2.putText(img, f'Accuracy: {accuracy:.2f}%', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Display the video frame
    cv2.imshow('Video', img)

    # Exit the loop when 'ESC' key is pressed
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()